# DB2-Salesforce connector: Basic user information updates

%load_ext autoreload
%autoreload 2

In [1]:
# Parameters
hours_range = 24*365*1 # number of hours to look back

# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# login credentials
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"mathisfun992728tXYwCb5yMm6XtohbTRw8ZdBbi",    
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",


In [2]:
import sys
#sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')

import pandas as pd
import datetime

## Obtain contacts from DB2 
that have last visit date within range of interest specified by day_range

In [3]:
# Depending on the task, use different query and cutoff

# Hourly update for new registrations
date_cutoff = (datetime.datetime.today().date() - datetime.timedelta(hours=hours_range))\
                    .strftime('%Y-%m-%d')
sql_query = "select id, name, username, block, email, sendEmail, registerDate, lastvisitDate \
                    from jos_users where registerDate >= '%s'" % date_cutoff

# display
print(sql_query)

select id, name, username, block, email, sendEmail, registerDate, lastvisitDate                     from jos_users where registerDate >= '2019-08-18'


In [4]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))
df = pd.read_sql_query(sql_query, engine)

# get user profile details
profile_df = pd.read_sql_query("select * from jos_user_profiles where profile_key in ('orgtype', 'organization', 'orcid')", engine)

In [5]:
# display
display(df.head(1))

display(profile_df.head(1))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate
0,258090,Igo HENRIQUE SILVA NUNES,igo,0,igo@prof.unipar.br,-1,2019-08-18 04:02:56,2019-08-18 04:02:56


,id,user_id,profile_key,profile_value,ordering,access
0,1,15623,orgtype,university,2,5


In [6]:
df = pd.merge(df, profile_df[profile_df['profile_key'] == 'orgtype'][['user_id', 'profile_value']],\
                         how='left', left_on='id', right_on='user_id')\
    .drop(columns=['user_id']).rename(columns={'profile_value':'orgtype'})

df = pd.merge(df, profile_df[profile_df['profile_key'] == 'organization'][['user_id', 'profile_value']],\
                         how='left', left_on='id', right_on='user_id')\
    .drop(columns=['user_id']).rename(columns={'profile_value':'organization'})
display(df.head(1))

df = pd.merge(df, profile_df[profile_df['profile_key'] == 'orcid'][['user_id', 'profile_value']],\
                         how='left', left_on='id', right_on='user_id')\
    .drop(columns=['user_id']).rename(columns={'profile_value':'orcid'})

# display
display(df.head(1))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization
0,258090,Igo HENRIQUE SILVA NUNES,igo,0,igo@prof.unipar.br,-1,2019-08-18 04:02:56,2019-08-18 04:02:56,universityfaculty,NaN


,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid
0,258090,Igo HENRIQUE SILVA NUNES,igo,0,igo@prof.unipar.br,-1,2019-08-18 04:02:56,2019-08-18 04:02:56,universityfaculty,NaN,NaN


In [7]:
# Obtain company domain information from DB2
domain_df = pd.read_sql_query("select name as domain_name, domain, industry, `size range` as size, country \
from wang159_myrmekes.companies_email_domain", engine)

In [8]:
# make sure domain is unique and drop NaN
domain_df = domain_df.drop_duplicates(subset='domain', keep='first').dropna()

In [9]:
# get email domain of nanoHUB users
def get_domain(this_email):
    seg_list = this_email.split('@')
    
    if len(seg_list) == 2:
        return seg_list[1].lower()
    else:
        return None
    
df['email_host'] = df['email'].apply(get_domain)

In [10]:
# save a constuct for debugging
from copy import deepcopy
temp = deepcopy(df)

In [11]:
# get domain by nanoHUB organization field
df = deepcopy(temp)
display(df.head(2))
df['organization'] = df['organization'].str.lower()
df = df.fillna('   ')

df.loc[(df['organization']=='nan'),'organization'] = None
df.loc[(df['organization']=='NaN'),'organization'] = None

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host
0,258090,Igo HENRIQUE SILVA NUNES,igo,0,igo@prof.unipar.br,-1,2019-08-18 04:02:56,2019-08-18 04:02:56,universityfaculty,NaN,NaN,prof.unipar.br
1,258091,Kenneth Henrik Olsen,radarblue,0,kenneth.olsen77@gmail.com,1,2019-08-18 04:03:35,2019-08-18 04:04:26,other,NaN,NaN,gmail.com


display(df.tail())

In [12]:
import nltk
#nltk.download('stopwords')

from nltk.corpus import stopwords
s_words = stopwords.words('english')

In [13]:
import re

def clean_domain(this_domain_name):
    
    if not this_domain_name:
        return None
    
    # Replace all non-alphanumeric characters with space
    this_domain_name = re.sub("[^0-9a-zA-Z]+", " ", this_domain_name.lower())
    name_list = this_domain_name.split(' ')
        
    # remove all stop words
    name_list = ['' if x in s_words else x for x in name_list]
    
    return set(filter(None, name_list))

In [14]:
display(domain_df.head(2))

,domain_name,domain,industry,size,country
0,strategic interns,strategicinterns.com,education management,1-10,india
1,bosque y comunidad,bosqueycomunidad.org,non-profit organization management,1-10,spain


In [15]:
# clean domain name
domain_df['domain_cleaned_set'] = domain_df.domain_name.apply(clean_domain)
display(domain_df.head(1))
# clean nanohub org name
df['org_cleaned_set'] = df.organization.apply(clean_domain)
display(df.head(1))

,domain_name,domain,industry,size,country,domain_cleaned_set
0,strategic interns,strategicinterns.com,education management,1-10,india,"{interns, strategic}"


,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set
0,258090,Igo HENRIQUE SILVA NUNES,igo,0,igo@prof.unipar.br,-1,2019-08-18 04:02:56,2019-08-18 04:02:56,universityfaculty,,,prof.unipar.br,{}


In [16]:
# hash cleaned domain name
domain_df['domain_cleaned_hash'] = domain_df['domain_cleaned_set'].apply(lambda x: '-'.join(sorted(list(x))) if x else None)

# hash cleaned clean nanohub org name
df['org_cleaned_hash'] = df['org_cleaned_set'].apply(lambda x: '-'.join(sorted(list(x))) if x else None)

In [17]:
display(domain_df.head(1))
display(df.head(1))

,domain_name,domain,industry,size,country,domain_cleaned_set,domain_cleaned_hash
0,strategic interns,strategicinterns.com,education management,1-10,india,"{interns, strategic}",interns-strategic


,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash
0,258090,Igo HENRIQUE SILVA NUNES,igo,0,igo@prof.unipar.br,-1,2019-08-18 04:02:56,2019-08-18 04:02:56,universityfaculty,,,prof.unipar.br,{},None


## Collaborative filtering for commerical email hosts

In [18]:
def predict_if_commerical(this_email_host_df):
    # predict if this email host is commerical
    top_cnt = this_email_host_df['org_cleaned_hash'].value_counts() #returns uniques in descending order
    
    if top_cnt.shape[0] > 0:
        if (top_cnt.iloc[0]/top_cnt.sum()) < 0.2:
            return True
        else:
            return False
    
    else:
        return False
    
    
is_email_commercial = df[['email_host', 'org_cleaned_hash']].groupby('email_host').apply(predict_if_commerical)

In [19]:
temp2 = deepcopy(df)

In [55]:
df = deepcopy(temp2)

In [56]:
# mark commerical email addresses
df = pd.merge(df, is_email_commercial.reset_index(name='is_email_commercial'),\
         how='left', left_on='email_host', right_on='email_host' ,)

In [57]:
display(df.tail(1))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash,is_email_commercial
28398,297108,Domenico Innarella,casy92,0,domenico.innarella@gmail.com,1,2020-08-17 23:28:47,2020-08-17 23:30:20,other,,,gmail.com,{},None,True


In [58]:
# obtain domain estimation from email addresses
df = pd.merge(df, domain_df[~domain_df.domain.isin(is_email_commercial[is_email_commercial].index)]['domain'],\
         how='left', left_on='email_host', right_on='domain').rename(columns={'domain':'domain_by_email'})

# display
display(df.head(2).T)

,0,1
id,258090,258091
name,Igo HENRIQUE SILVA NUNES,Kenneth Henrik Olsen
username,igo,radarblue
block,0,0
email,igo@prof.unipar.br,kenneth.olsen77@gmail.com
sendEmail,-1,1
registerDate,2019-08-18 04:02:56,2019-08-18 04:03:35
lastvisitDate,2019-08-18 04:02:56,2019-08-18 04:04:26
orgtype,universityfaculty,other
organization,,


## Fuzzy derive organization from nanoHUB profiles

In [59]:
# attempt direct join by hash
domain_subset_df = domain_df[domain_df.domain.isin(df.email_host.unique())][['domain', 'domain_cleaned_hash']]
# only take values that exist
domain_subset_df = domain_subset_df[~domain_subset_df.domain.isna()&~domain_subset_df.domain_cleaned_hash.isna()]

derived_df = pd.merge(df, domain_subset_df.rename(columns={'domain':'domain_by_profile'}),\
         how='left', left_on='org_cleaned_hash', right_on='domain_cleaned_hash')\
        .drop('domain_cleaned_hash', axis=1)

# display
derived_df.sample(5).T

,18268,17136,16593,5698,24366
id,284689,283319,282688,267731,292176
name,Jessie Hennessy,David García Allo,kumar k,Jeremy Treiber,霍非
username,jessie.hennessy,davidga568,pankajiitb97,jatreibe,hf326267119
block,0,0,0,0,0
email,jessie.hennessy@ucdconnect.ie,davidgarciaallo43@gmail.com,pankajiitb97@gmail.com,jatreibe@ucsd.edu,hf326267119@gmail.com
sendEmail,-1,-1,-1,-1,1
registerDate,2020-04-15 17:55:52,2020-04-06 14:54:24,2020-04-01 04:29:14,2019-10-04 17:40:36,2020-06-21 10:29:12
lastvisitDate,2020-04-19 11:42:00,2020-05-09 15:06:02,2020-04-01 04:29:15,2019-10-18 21:32:56,2020-06-21 14:02:49
orgtype,universitygraduate,universityfaculty,universitygraduate,universityundergraduate,universitygraduate
organization,,,,,


In [60]:
# attempt to reduce domain_df size to speedup Jaccard calculation
from itertools import chain

all_word_set = set(chain.from_iterable(derived_df[(derived_df['domain_by_profile'].isna() & (~derived_df['organization'].isna()))]\
                    .org_cleaned_set.values))

# select the domain_df entry with words from all_word_set
domain_subset_df = domain_df[domain_df.domain_cleaned_set.notnull()]
domain_subset_df = domain_subset_df[domain_subset_df.domain_cleaned_set\
                             .apply(lambda x: True if len(x.intersection(all_word_set)) > 0 else False)]

In [61]:
display(domain_subset_df.head(2))

,domain_name,domain,industry,size,country,domain_cleaned_set,domain_cleaned_hash
4,"cardinal strategies, llc",cardinal-strategies.com,civil engineering,11-50,united states,"{llc, strategies, cardinal}",cardinal-llc-strategies
16,grail research,grailresearch.com,research,201-500,united states,"{grail, research}",grail-research


In [62]:
def get_org(this_df):
    # look at the cohort with this label, do they have a commonly agreed domain_by_email? If so, use it
    this_dbe_count = this_df.domain_by_email.value_counts()

    if this_dbe_count.shape[0] < 1:
        return None

    this_top_dbe_perc = this_dbe_count.iloc[0]/this_dbe_count.sum()

    # if top choice occupies majority of cases, then this cohort can be implied to this domain_by_email address
    if this_top_dbe_perc > 0.5:
        return this_dbe_count.index[0] 


derived_hash = derived_df.groupby('org_cleaned_hash').apply(get_org)

In [63]:
derived_df = pd.merge(derived_df, derived_hash.reset_index(name='domain_by_profile_infer')\
                      , how='left', left_on='org_cleaned_hash', right_on='org_cleaned_hash')

In [64]:
# calculate a composite domain based on domain_by_email, domain_by_profile, and domain_by_profile_infer
def get_composite_domain(this_df):

    # if domain found via profile with direct match, use it
    if this_df.domain_by_profile != '':
        return this_df.domain_by_profile
    
    # if domain found via email, use it
    if this_df.domain_by_email != '':
        return this_df.domain_by_email    
    
    # if domain found via profile but not with a direct match, use it
    if this_df.domain_by_profile_infer != '':
        return this_df.domain_by_profile_infer
    
    # else, no organization can be concluded, return none
    return None

    
derived_df['domain_final'] = derived_df[['domain_by_email', 'domain_by_profile', 'domain_by_profile_infer']].fillna('')\
                                                        .apply(get_composite_domain, axis=1)

In [65]:
display(derived_df.head(2))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash,is_email_commercial,domain_by_email,domain_by_profile,domain_by_profile_infer,domain_final
0,258090,Igo HENRIQUE SILVA NUNES,igo,0,igo@prof.unipar.br,-1,2019-08-18 04:02:56,2019-08-18 04:02:56,universityfaculty,,,prof.unipar.br,{},None,False,NaN,NaN,NaN,None
1,258091,Kenneth Henrik Olsen,radarblue,0,kenneth.olsen77@gmail.com,1,2019-08-18 04:03:35,2019-08-18 04:04:26,other,,,gmail.com,{},None,True,NaN,NaN,NaN,None


## Update Salesforce institution 

In [66]:
# get a unique list of organizations that appears in contact
#org_df = derived_df[['domain_name', 'domain', 'industry', 'size', 'country']].drop_duplicates().dropna()
org_df = pd.merge(derived_df[['domain_final']].drop_duplicates().dropna(), domain_df, how='inner', left_on='domain_final', right_on='domain') 
org_df = org_df[['domain_name', 'domain', 'industry', 'size', 'country']]

# display
display(org_df.head(2))

,domain_name,domain,industry,size,country
0,purdue university,purdue.edu,higher education,10001+,united states
1,zhejiang university,zju.edu.cn,higher education,5001-10000,china


In [67]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Name'] = org_df['domain_name']

df_sf['Size_Range__c'] = org_df['size'].fillna('Unknown')
df_sf['Country__c'] = org_df['country']
df_sf['Domain__c'] = org_df['domain']
df_sf['Industry__c'] = org_df['industry']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Name,purdue university,zhejiang university
Size_Range__c,10001+,5001-10000
Country__c,united states,china
Domain__c,purdue.edu,zju.edu.cn
Industry__c,higher education,higher education


In [68]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

db_s.object_id = 'organization__c'
db_s.external_id = 'Domain__c'

Obtained Salesforce access token ...... True


In [69]:
display(df_sf.head(2))
display(df_sf.tail(2))


,Name,Size_Range__c,Country__c,Domain__c,Industry__c
0,purdue university,10001+,united states,purdue.edu,higher education
1,zhejiang university,5001-10000,china,zju.edu.cn,higher education


,Name,Size_Range__c,Country__c,Domain__c,Industry__c
854,"colégio jean piaget - santos - sp, brazil",51-200,brazil,jeanpiaget.g12.br,education management
855,jerusalem college of engineering - india,11-50,india,jerusalemengg.ac.in,education management


In [70]:
## correcting name overflows
names = df_sf['Name'].str.len()

sf_char_limit = 80
name_indexes = names.index[names >= sf_char_limit].tolist()
for index in name_indexes:
    df_sf['Name'][index] =  df_sf['Name'][index][0:sf_char_limit]

In [71]:
display(df_sf.head(2))

,Name,Size_Range__c,Country__c,Domain__c,Industry__c
0,purdue university,10001+,united states,purdue.edu,higher education
1,zhejiang university,5001-10000,china,zju.edu.cn,higher education


In [72]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000Op9QhAAJ
hello
[Success] CSV upload successful. Job ID = 7505w00000Op9QhAAJ
[Success] Closing job successful. Job ID = 7505w00000Op9QhAAJ


In [73]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000Op9QhAAJ',
 'operation': 'upsert',
 'object': 'organization__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-08-18T00:05:33.000+0000',
 'systemModstamp': '2020-08-18T00:05:34.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'Domain__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [74]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''


## Obtain newly updated organization IDs from Salesforce

In [75]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [76]:
# get Salesforce ID for organizations
sf_org_ID_df = db_s.query_data('SELECT Id, Domain__c FROM organization__c')

# display
sf_org_ID_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000Op9TCAAZ
{"id":"7505w00000Op9TCAAZ","operation":"query","object":"organization__c","createdById":"0055w00000DM5bOAAT","createdDate":"2020-08-18T00:06:47.000+0000","systemModstamp":"2020-08-18T00:06:47.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000Op9TCAAZ","operation":"query","object":"organization__c","createdById":"0055w00000DM5bOAAT","createdDate":"2020-08-18T00:06:47.000+0000","systemModstamp":"2020-08-18T00:06:48.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":6257,"retries":0,"totalProcessingTime":374}
[Success] Bulk job completed successfully.


,Domain__c,Id
0,upenn.edu,a0r5w00000V42c0AAB
1,fer.hr,a0r5w00000V42c1AAB
2,wheatoncollege.edu,a0r5w00000V42c2AAB


In [77]:
# join salesforce ID back to contact DF
df = pd.merge(sf_org_ID_df, derived_df, how='right', left_on='Domain__c', right_on='domain_final')\
            .drop('Domain__c', axis=1)\
            .rename(columns={'Id':'Salesforce_org_ID'})

In [78]:
# display
display(df.head(2))

,Salesforce_org_ID,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash,is_email_commercial,domain_by_email,domain_by_profile,domain_by_profile_infer,domain_final
0,a0r5w00000V42c0AAB,268121,"Thiruvengadam, Rangarajan Narasimhan",nitin17,0,nitin17@upenn.edu,0,2019-10-09 16:00:08,2019-10-09 16:00:08,universitygraduate,,,upenn.edu,{},None,False,upenn.edu,NaN,NaN,upenn.edu
1,a0r5w00000V42c0AAB,273295,"Cai, Yiyu",caiyiyu,0,caiyiyu@seas.upenn.edu,0,2019-12-08 23:30:54,2020-03-27 22:12:53,universitystaff,university of pennsylvania,,seas.upenn.edu,"{university, pennsylvania}",pennsylvania-university,False,NaN,upenn.edu,None,upenn.edu


## Match data with Salesforce format

In [79]:
# split full name into first, middle, and last names
def split_full_name(this_name):
    this_name_list = list(filter(None, this_name.split(' ')))
    
    if len(this_name_list) == 1:
        # single word name
        return pd.Series([this_name_list[0],None, this_name_list[0]])
    
    elif len(this_name_list) > 1:
        # multi word name
        return pd.Series([this_name_list[0],' '.join(this_name_list[1:-1]),this_name_list[-1]])

In [81]:
## stop here
display(df.head(1))

,Salesforce_org_ID,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash,is_email_commercial,domain_by_email,domain_by_profile,domain_by_profile_infer,domain_final
0,a0r5w00000V42c0AAB,268121,"Thiruvengadam, Rangarajan Narasimhan",nitin17,0,nitin17@upenn.edu,0,2019-10-09 16:00:08,2019-10-09 16:00:08,universitygraduate,,,upenn.edu,{},None,False,upenn.edu,NaN,NaN,upenn.edu


In [83]:
## if last visit date is empty - set it equal to the register date
lvd = df.lastvisitDate.index[df.lastvisitDate == '   '].tolist()

for i in lvd:
    df.iloc[i,8] = df.iloc[i,7]

display(df.head(2))

,Salesforce_org_ID,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash,is_email_commercial,domain_by_email,domain_by_profile,domain_by_profile_infer,domain_final
0,a0r5w00000V42c0AAB,268121,"Thiruvengadam, Rangarajan Narasimhan",nitin17,0,nitin17@upenn.edu,0,2019-10-09 16:00:08,2019-10-09 16:00:08,universitygraduate,,,upenn.edu,{},None,False,upenn.edu,NaN,NaN,upenn.edu
1,a0r5w00000V42c0AAB,273295,"Cai, Yiyu",caiyiyu,0,caiyiyu@seas.upenn.edu,0,2019-12-08 23:30:54,2020-03-27 22:12:53,universitystaff,university of pennsylvania,,seas.upenn.edu,"{university, pennsylvania}",pennsylvania-university,False,NaN,upenn.edu,None,upenn.edu


In [84]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf[['firstname', 'Middle_name__c', 'lastname']] = df['name'].apply(split_full_name)

df_sf['nanoHUB_user_ID__c']            = df['id']
df_sf['nanoHUB_username__c']           = df['username']
df_sf['Email']                         = df['email'].fillna('').apply(lambda x: '' if '@invalid' in x else x)

# for sendEmail: 0 = opt-out, 1 = receive email. For salesforce HasOptedOutOfEmail, it's exact opposite
df_sf['HasOptedOutOfEmail']            = df['sendEmail'].apply(lambda x: 0 if x==1 else 1)
df_sf['nanoHUB_account_BLOCKED__c']    = df['block'].fillna(0)

# solidify time-related columns from datetime to string
df_sf['nanoHUB_registration_date__c']  = df['registerDate'].dt.date.fillna('').astype('str')
df_sf['nanoHUB_last_active_date__c']   = df['lastvisitDate'].dt.date.fillna('').astype('str')

# Tableau detailed view
df_sf['Detailed_user_timeline_to_Tableau__c'] = df_sf['Email'].apply(lambda x: 'https://tableauqa.itap.purdue.edu/views/profile/ProfileTimeline?Id%20Email='+x+'\
&:iframeSizedToWindow=true&:embed=y&:showAppBanner=false\
&:display_count=no&:showVizHome=no#6' if x != '' else '')

# nanoHUB user profile
df_sf['nanoHUB_user_page__c'] = df_sf['nanoHUB_user_ID__c'].apply(lambda x: 'https://nanohub.org/members/%d'%x if x != '' else '')

df_sf['ORCID__c'] = df['orcid'].fillna('')
df_sf['Organization__c'] = df['organization'].fillna('')

# derived information
df_sf['Organization_email_derived__c'] = df['Salesforce_org_ID']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
firstname,"Thiruvengadam,","Cai,"
Middle_name__c,Rangarajan,
lastname,Narasimhan,Yiyu
nanoHUB_user_ID__c,268121,273295
nanoHUB_username__c,nitin17,caiyiyu
Email,nitin17@upenn.edu,caiyiyu@seas.upenn.edu
HasOptedOutOfEmail,1,1
nanoHUB_account_BLOCKED__c,0,0
nanoHUB_registration_date__c,2019-10-09,2019-12-08
nanoHUB_last_active_date__c,2019-10-09,2020-03-27


In [1]:
## filtering out accounts that have positive block attributes
spam_rows = df_sf[df_sf['nanoHUB_account_BLOCKED__c'].astype(str).str.contains("1")]
good_rows = df_sf[~df_sf['nanoHUB_account_BLOCKED__c'].astype(str).str.contains('1')]

NameError: name 'df_sf' is not defined

## To Salesforce Sales Cloud CRM

In [136]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [2]:
display(spam_rows.head(2))
display(good_rows.tail(2))

NameError: name 'spam_rows' is not defined

In [139]:
# send data to Salesforce
db_s.send_data(good_rows)

[Success] Bulk job creation successful. Job ID = 7505w00000OpBXkAAN
hello
[Success] CSV upload successful. Job ID = 7505w00000OpBXkAAN
[Success] Closing job successful. Job ID = 7505w00000OpBXkAAN


In [140]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000OpBXkAAN',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-08-18T01:56:28.000+0000',
 'systemModstamp': '2020-08-18T01:56:43.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [141]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''


In [142]:
display(df_sf.tail(2))

,firstname,Middle_name__c,lastname,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c,Detailed_user_timeline_to_Tableau__c,nanoHUB_user_page__c,ORCID__c,Organization__c,Organization_email_derived__c
28472,Nikolai,,H,297107,honikolai,honikolai@gmail.com,1,0,2020-08-17,2020-08-17,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/297107,,,NaN
28473,Domenico,,Innarella,297108,casy92,domenico.innarella@gmail.com,0,0,2020-08-17,2020-08-17,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/297108,,,NaN


## Verification
Checking to see if salesforce has updated contacts with the new users

request name and nanohub_user_id

In [143]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [144]:
# get Salesforce ID for organizations
sf_org_ID_df = db_s.query_data('SELECT Id, firstname, lastname, nanoHUB_user_ID__c FROM Contact')

# display
sf_org_ID_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000OpBYoAAN
{"id":"7505w00000OpBYoAAN","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-08-18T02:24:11.000+0000","systemModstamp":"2020-08-18T02:24:11.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000OpBYoAAN","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-08-18T02:24:11.000+0000","systemModstamp":"2020-08-18T02:24:11.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000OpBYoAAN","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDat

,FirstName,Id,LastName,nanoHUB_user_ID__c
0,hubrepo,0035w000031Vsp1AAC,hubrepo,998.0
1,nanoHUB,0035w000031Vsp2AAC,support,1683.0
2,Grid,0035w000031Vsp3AAC,Statistics,1684.0


In [146]:
display(sf_org_ID_df.tail(5))

,FirstName,Id,LastName,nanoHUB_user_ID__c
225004,Flavio,0035w00003BRVPJAA5,Dantas,296315.0
225005,truss,0035w00003BRVPKAA5,rental,296317.0
225006,Mingyu,0035w00003BRVPLAA5,Gong,296319.0
225007,Ganesh,0035w00003BRVPMAA5,Chalamalasetti,296321.0
225008,Nolan,0035w00003BRVPNAA5,Parker,296325.0
